In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import pyarrow as pa
import pyarrow.parquet as pq
import gcsfs
import getpass
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
import geopandas as gpd

# import sgis as sg
import dapla as dp
import datetime
from dapla.auth import AuthClient
from dapla import FileClient
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import requests
from pyjstat import pyjstat
from klass import search_classification


import sys

sys.path.append("../functions")
import kommune_pop
import kommune_inntekt
import kpi
import ao
import kommune_translate

fs = FileClient.get_gcs_file_system()
import numpy as np


import warnings

warnings.filterwarnings("ignore")

import math

# good_df = ao.rette_bedrifter(good_df)

In [ ]:
# search_classification()

In [ ]:
year = "2020"
fjor = "2019"
limit = 0.6

In [ ]:
fil_path = [
    f
    for f in fs.glob(
        f"gs://ssb-prod-noeku-data-produkt/eimerdb/nokubasen/skjemadata/aar={year}/skjema=RA-0174-1/*"
    )
    if f.endswith(".parquet")
]

# Use the ParquetDataset to read multiple files
dataset = pq.ParquetDataset(fil_path, filesystem=fs)
table = dataset.read()

# Convert to Pandas DataFrame
skjema = table.to_pandas()

In [ ]:
felt_id_values = [
    "V_ORGNR",
    "F_ADRESSE",
    "FJOR_NACE_B_T1",
    "TMP_SN2007_5",
    "B_KOMMUNENR",
    "REGTYPE",
    "B_SYSSELSETTING_SYSS",
    "TMP_NY_BDR_SYSS",
    "GJELDENDE_BDR_SYSS",
    "FJOR_SYSSEL_T1",
    "LONN_PST_AORDN",
    "GJELDENDE_LONN_KR",
    "LONN",
    "FJOR_LONN_KR_T1",
    "TMP_SNITTLONN",
    "FJOR_SNITTLONN_T1",
    "GJELDENDE_OMSETN_KR",
    "OMSETN_KR",
    "FJOR_OMSETN_KR_T1",
    "TMP_SNITTOMS",
    "FJOR_SNITTOMS_T1",
    "TMP_SALGSINT_BED",
    "TMP_FORBRUK_BED",
    "VAREKOST_BED",
    "GJELDENDE_DRIFTSK_KR",
    "DRIFTSKOST_KR",
    "FJOR_DRIFTSKOST_KR_T1",
    "NACEF_5",
    "SALGSINT",
    "FORBRUK",
    "TMP_NO_P4005",
    "TMP_AVPROS_ORGFORB",
    "ORGNR_N_1",
    "TMP_NO_OMSETN",
    "TMP_DRIFTSKOSTNAD_9010",
    "TMP_DRIFTSKOSTNAD_9910",
]

# Assuming `skjema` is your DataFrame and `felt_id_values` is your list of values
skjema = skjema[skjema["feltnavn"].isin(felt_id_values)]

In [ ]:
pivot_df = skjema.pivot_table(
    index=["id", "radnr", "lopenr"],
    columns="feltnavn",
    values="feltverdi",
    aggfunc="first",
)
pivot_df = pivot_df.reset_index()
pivot_df.columns = pivot_df.columns.str.lower()

pivot_df["year"] = year

pivot_df

In [ ]:
bedrift = pivot_df.loc[pivot_df["radnr"] > 0]

In [ ]:
bedrift["year"] = year

bedrift["new_oms"] = bedrift["gjeldende_omsetn_kr"]

In [ ]:
# Assuming 'bedrift' is your DataFrame
columns_to_fill = ["omsetn_kr", "driftskost_kr"]

# Convert columns to numeric, replacing non-convertible values with NaN
bedrift[columns_to_fill] = bedrift[columns_to_fill].apply(
    pd.to_numeric, errors="coerce"
)

# Fill NaN values with 0 for the specified columns
bedrift[columns_to_fill] = bedrift[columns_to_fill].fillna(0)


# hjelpe virksomheter
if_condition = bedrift["regtype"] == "04"

# If the condition is True, set 'omsetn_kr' equal to 'driftskost_kr'
bedrift.loc[if_condition, "omsetn_kr"] = bedrift.loc[if_condition, "driftskost_kr"]


# Group by 'id' and calculate the sum
grouped_bedrift = (
    bedrift.groupby("id")[["omsetn_kr", "driftskost_kr"]].sum().reset_index()
)

# Rename the columns
grouped_bedrift.rename(
    columns={"omsetn_kr": "tot_oms_fordelt", "driftskost_kr": "tot_driftskost_fordelt"},
    inplace=True,
)

# Merge the grouped DataFrame back to the original DataFrame based on 'id'
bedrift = pd.merge(bedrift, grouped_bedrift, on="id", how="left")

In [ ]:
bedrift.head()

In [ ]:
kommune_befolk = kommune_pop.befolkning_behandling(year, fjor)
kommune_inn = kommune_inntekt.inntekt_behandling(year, fjor)
kpi_df = kpi.process_kpi_data(year)

In [ ]:
kommune_befolk.head()

In [ ]:
kommune_inn.head()

In [ ]:
kpi_df.head()

In [ ]:
# Convert string columns to numeric
bedrift["gjeldende_bdr_syss"] = pd.to_numeric(
    bedrift["gjeldende_bdr_syss"], errors="coerce"
)
bedrift["fjor_syssel_t1"] = pd.to_numeric(bedrift["fjor_syssel_t1"], errors="coerce")

# Perform division after conversion
bedrift["emp_delta"] = bedrift["gjeldende_bdr_syss"] / bedrift["fjor_syssel_t1"]

In [ ]:
imputable_df = bedrift.copy()

imputable_df["n4"] = imputable_df["nacef_5"].str[:5]

imputable_df = pd.merge(imputable_df, kommune_befolk, on="b_kommunenr", how="left")
imputable_df = pd.merge(imputable_df, kommune_inn, on="b_kommunenr", how="left")
imputable_df = pd.merge(imputable_df, kpi_df, on="n4", how="left")

In [ ]:
# Ensure columns are numeric
imputable_df["fjor_omsetn_kr_t1"] = pd.to_numeric(
    imputable_df["fjor_omsetn_kr_t1"], errors="coerce"
)
imputable_df["inflation_rate"] = pd.to_numeric(
    imputable_df["inflation_rate"], errors="coerce"
)
imputable_df["befolkning_delta"] = pd.to_numeric(
    imputable_df["befolkning_delta"], errors="coerce"
)
imputable_df["emp_delta"] = pd.to_numeric(imputable_df["emp_delta"], errors="coerce")
imputable_df["inntekt_delta"] = pd.to_numeric(
    imputable_df["inntekt_delta"], errors="coerce"
)

general_inflation_rate = imputable_df.loc[
    imputable_df["n4"] == "47.78", "inflation_rate"
].values[0]
imputable_df["inflation_rate"] = imputable_df["inflation_rate"].fillna(
    general_inflation_rate
)

imputable_df["inflation_rate_oms"] = (
    imputable_df["fjor_omsetn_kr_t1"] * imputable_df["inflation_rate"]
)
imputable_df["befolkning_delta_oms"] = (
    imputable_df["fjor_omsetn_kr_t1"] * imputable_df["befolkning_delta"]
)
imputable_df["emp_delta_oms"] = (
    imputable_df["fjor_omsetn_kr_t1"] * imputable_df["emp_delta"]
)
imputable_df["inntekt_delta_oms"] = (
    imputable_df["fjor_omsetn_kr_t1"] * imputable_df["inntekt_delta"]
)

# imputable_df['inflation_rate_oms'] = imputable_df['inflation_rate_oms'].round(0).astype(int)
# imputable_df['befolkning_delta_oms'] = imputable_df['befolkning_delta_oms'].round(0).astype(int)
# imputable_df['emp_delta_oms'] = imputable_df['emp_delta_oms'].round(0).astype(int)
# imputable_df['inntekt_delta_oms'] = imputable_df['inntekt_delta_oms'].round(0).astype(int)


imputable_df.head()

In [ ]:
imputable_df.head(10)

In [ ]:
year

In [ ]:
fjor

In [ ]:
import sys

sys.path.append("../functions")

import kommune_translate

In [ ]:
year_int = int(year)
fjor_int = int(fjor)
print(year_int)
print(fjor_int)

In [ ]:
df1 = kommune_inntekt.fetch_inntekt_data(year)
df2 = kommune_inntekt.fetch_inntekt_data(fjor)

df1 = df1.dropna(subset=["value"])
df2 = df2.dropna(subset=["value"])

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
year_int = int(year)
fjor_int = int(fjor)

In [ ]:
df1["year"] = year_int
df2["year"] = fjor_int

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
kommune_int_aar = df1.rename(columns={"value": "inntekt"})
kommune_int_fjor = df2.rename(columns={"value": "inntektx"})

In [ ]:
kommune_int_aar.head()

In [ ]:
kommune_int_fjor.head()

In [ ]:
if df1["year"].iloc[0] < 2020:
    df1 = kommune_translate.translate_kommune_kodes(df1)

In [ ]:
df2["kommune_nr"].value_counts()

In [ ]:
pd.set_option("display.max_rows", None)
# df2['kommune_nr'].value_counts()

In [ ]:
# Check if the year is less than 2020 before translating
if df1["year"].iloc[0] < 2020:
    df1 = kommune_translate.translate_kommune_kodes(df1)

In [ ]:
df1.head()

In [ ]:
pd.set_option("display.max_rows", None)
# df1['kommune_nr'].value_counts()

In [ ]:
if df2["year"].iloc[0] < 2020:
    df2 = kommune_translate.translate_kommune_kodes(df2)

In [ ]:
df2.head()

In [ ]:
# Confirm the data type of the 'year' column
print("Data type of 'year' column:", df2["year"].dtype)

# Print the first value of the 'year' column
print("First value of 'year' column:", df2["year"].iloc[0])

# Check the condition
if df2["year"].iloc[0] < 2020:
    print("Condition is true, translating kommune codes...")
    df2 = kommune_translate.translate_kommune_kodes(df2)
else:
    print("Condition is false, not translating kommune codes.")

In [ ]:
merged_kommunes = {
    "0101": ["3001"],
    "0104": ["3002"],
    "0105": ["3003"],
    "0106": ["3004"],
    "0111": ["3011"],
    "0118": ["3012"],
    "0119": ["3013"],
    "0121": ["3026"],
    "0122": ["3014"],
    "0123": ["3014"],
    "0124": ["3014"],
    "0125": ["3014"],
    "0127": ["3015"],
    "0128": ["3016"],
    "0135": ["3017"],
    "0136": ["3002"],
    "0137": ["3018"],
    "0138": ["3014"],
    "0211": ["3019"],
    "0213": ["3020"],
    "0214": ["3021"],
    "0215": ["3022"],
    "0216": ["3023"],
    "0217": ["3020"],
    "0219": ["3024"],
    "0220": ["3025"],
    "0221": ["3026"],
    "0226": ["3030"],
    "0227": ["3030"],
    "0228": ["3027"],
    "0229": ["3028"],
    "0230": ["3029"],
    "0231": ["3030"],
    "0233": ["3031"],
    "0234": ["3032"],
    "0235": ["3033"],
    "0236": ["3034"],
    "0237": ["3035"],
    "0238": ["3036"],
    "0239": ["3037"],
    "0402": ["3401"],
    "0403": ["3403"],
    "0412": ["3411"],
    "0415": ["3412"],
    "0417": ["3413"],
    "0418": ["3414"],
    "0419": ["3415"],
    "0420": ["3416"],
    "0423": ["3417"],
    "0425": ["3418"],
    "0426": ["3419"],
    "0427": ["3420"],
    "0428": ["3421"],
    "0429": ["3422"],
    "0430": ["3423"],
    "0432": ["3424"],
    "0434": ["3425"],
    "0436": ["3426"],
    "0437": ["3427"],
    "0438": ["3428"],
    "0439": ["3429"],
    "0441": ["3430"],
    "0501": ["3405"],
    "0502": ["3407"],
    "0511": ["3431"],
    "0512": ["3432"],
    "0513": ["3433"],
    "0514": ["3434"],
    "0515": ["3435"],
    "0516": ["3436"],
    "0517": ["3437"],
    "0519": ["3438"],
    "0520": ["3439"],
    "0521": ["3440"],
    "0522": ["3441"],
    "0528": ["3442"],
    "0529": ["3443"],
    "0532": ["3053"],
    "0533": ["3054"],
    "0534": ["3446"],
    "0536": ["3447"],
    "0538": ["3448"],
    "0540": ["3449"],
    "0541": ["3450"],
    "0542": ["3451"],
    "0543": ["3452"],
    "0544": ["3453"],
    "0545": ["3454"],
    "0602": ["3005"],
    "0604": ["3006"],
    "0605": ["3007"],
    "0612": ["3038"],
    "0615": ["3039"],
    "0616": ["3040"],
    "0617": ["3041"],
    "0618": ["3042"],
    "0619": ["3043"],
    "0620": ["3044"],
    "0621": ["3045"],
    "0622": ["3046"],
    "0623": ["3047"],
    "0624": ["3048"],
    "0625": ["3005"],
    "0626": ["3049"],
    "0627": ["3025"],
    "0628": ["3025"],
    "0631": ["3050"],
    "0632": ["3051"],
    "0633": ["3052"],
    "0701": ["3801"],
    "0704": ["3803"],
    "0710": ["3804"],
    "0711": ["3005"],
    "0712": ["3805"],
    "0713": ["3802"],
    "0715": ["3802"],
    "0716": ["3803"],
    "0729": ["3811"],
    "0805": ["3806"],
    "0806": ["3807"],
    "0807": ["3808"],
    "0811": ["3812"],
    "0814": ["3813"],
    "0815": ["3814"],
    "0817": ["3815"],
    "0819": ["3816"],
    "0821": ["3817"],
    "0822": ["3817"],
    "0826": ["3818"],
    "0827": ["3819"],
    "0828": ["3820"],
    "0829": ["3821"],
    "0830": ["3822"],
    "0831": ["3823"],
    "0833": ["3824"],
    "0834": ["3825"],
    "0901": ["4201"],
    "0904": ["4202"],
    "0906": ["4203"],
    "0911": ["4211"],
    "0912": ["4212"],
    "0914": ["4213"],
    "0919": ["4214"],
    "0926": ["4215"],
    "0928": ["4216"],
    "0929": ["4217"],
    "0935": ["4218"],
    "0937": ["4219"],
    "0938": ["4220"],
    "0940": ["4221"],
    "0941": ["4222"],
    "1001": ["4204"],
    "1002": ["4205"],
    "1003": ["4206"],
    "1004": ["4207"],
    "1014": ["4223"],
    "1017": ["4204"],
    "1018": ["4204"],
    "1021": ["4205"],
    "1026": ["4224"],
    "1027": ["4225"],
    "1029": ["4205"],
    "1032": ["4225"],
    "1034": ["4226"],
    "1037": ["4227"],
    "1046": ["4228"],
    "1102": ["1108"],
    "1103": ["1103"],
    "1129": ["1108"],
    "1141": ["1103"],
    "1142": ["1103"],
    "1201": ["4601"],
    "1211": ["4611"],
    "1216": ["4612"],
    "1219": ["4613"],
    "1221": ["4614"],
    "1222": ["4615"],
    "1223": ["4616"],
    "1224": ["4617"],
    "1227": ["4618"],
    "1228": ["4618"],
    "1231": ["4618"],
    "1232": ["4619"],
    "1233": ["4620"],
    "1234": ["4621"],
    "1235": ["4621"],
    "1238": ["4622"],
    "1241": ["4624"],
    "1242": ["4623"],
    "1243": ["4624"],
    "1244": ["4625"],
    "1245": ["4626"],
    "1246": ["4626"],
    "1247": ["4627"],
    "1251": ["4628"],
    "1252": ["4629"],
    "1253": ["4630"],
    "1256": ["4631"],
    "1259": ["4626"],
    "1260": ["4631"],
    "1263": ["4631"],
    "1264": ["4632"],
    "1265": ["4633"],
    "1266": ["4634"],
    "1401": ["4602"],
    "1411": ["4635"],
    "1412": ["4636"],
    "1413": ["4637"],
    "1416": ["4638"],
    "1417": ["4639"],
    "1418": ["4640"],
    "1419": ["4640"],
    "1420": ["4640"],
    "1421": ["4641"],
    "1422": ["4642"],
    "1424": ["4643"],
    "1426": ["4644"],
    "1428": ["4645"],
    "1429": ["4646"],
    "1430": ["4647"],
    "1431": ["4647"],
    "1432": ["4647"],
    "1433": ["4647"],
    "1438": ["4648"],
    "1439": ["4602"],
    "1441": ["4649"],
    "1443": ["4649"],
    "1444": ["1577"],
    "1445": ["4650"],
    "1449": ["4651"],
    "1502": ["1506"],
    "1504": ["1507"],
    "1514": ["1514"],
    "1519": ["1577"],
    "1523": ["1507"],
    "1524": ["1578"],
    "1526": ["1578"],
    "1529": ["1507"],
    "1534": ["1507"],
    "1543": ["1506"],
    "1545": ["1506"],
    "1546": ["1507"],
    "1548": ["1579"],
    "1551": ["1579"],
    "1571": ["5055"],
    "1805": ["1806"],
    "1826": ["1826"],
    "1849": ["1875"],
    "1852": ["5412"],
    "1853": ["1853"],
    "1854": ["1806"],
    "1867": ["1867"],
    "1902": ["5401"],
    "1903": ["5402"],
    "1911": ["5411"],
    "1913": ["5412"],
    "1917": ["5413"],
    "1919": ["5414"],
    "1920": ["5415"],
    "1922": ["5416"],
    "1923": ["5417"],
    "1924": ["5418"],
    "1925": ["5419"],
    "1926": ["5420"],
    "1927": ["5421"],
    "1928": ["5421"],
    "1929": ["5421"],
    "1931": ["5421"],
    "1933": ["5422"],
    "1936": ["5423"],
    "1938": ["5424"],
    "1939": ["5425"],
    "1940": ["5426"],
    "1941": ["5427"],
    "1942": ["5428"],
    "1943": ["5429"],
    "2002": ["5404"],
    "2003": ["5405"],
    "2004": ["5406"],
    "2011": ["5430"],
    "2012": ["5403"],
    "2014": ["5432"],
    "2015": ["5433"],
    "2017": ["5406"],
    "2018": ["5434"],
    "2019": ["5435"],
    "2020": ["5436"],
    "2021": ["5437"],
    "2022": ["5438"],
    "2023": ["5439"],
    "2024": ["5440"],
    "2025": ["5441"],
    "2027": ["5442"],
    "2028": ["5443"],
    "2030": ["5444"],
    "5001": ["5001"],
    "5004": ["5006"],
    "5005": ["5007"],
    "5011": ["5055"],
    "5013": ["5056"],
    "5015": ["5057"],
    "5016": ["5059"],
    "5017": ["5057"],
    "5018": ["5058"],
    "5019": ["5058"],
    "5023": ["5059"],
    "5024": ["5059"],
    "5025": ["5025"],
    "5030": ["5001"],
    "5039": ["5006"],
    "5040": ["5007"],
    "5048": ["5007"],
    "5050": ["5060"],
    "5051": ["5060"],
    "1850": ["1806", "1875"],
    "5012": ["5055", "5056", "5059"],
}

# Iterate over each 'kommune_nr' value in the DataFrame

df_test = df2.copy()

appended_rows = []

# Iterate over each 'kommune_nr' value in the DataFrame
for index, row in df_test.iterrows():
    kommune_nr = row["kommune_nr"]
    if kommune_nr in merged_kommunes:
        translated_values = merged_kommunes[kommune_nr]
        value = row["value"]
        num_translations = len(translated_values)
        if num_translations == 1:
            # If there's only one translation, replace 'kommune_nr' directly
            df_test.at[index, "kommune_nr"] = translated_values[0]
        else:
            # If there are multiple translations, distribute the value evenly
            value_per_translation = math.ceil(value / num_translations)
            for translation in translated_values:
                appended_rows.append(
                    {"kommune_nr": translation, "value": value_per_translation}
                )

# Append the rows to the DataFrame
new_df = pd.concat([df_test, pd.DataFrame(appended_rows)], ignore_index=True)

# # Ensure new_kommune_nr is hashable by converting to string (if needed)
# new_df['new_kommune_nr'] = new_df['new_kommune_nr'].apply(lambda x: str(x) if isinstance(x, list) else x)

# Aggregate values for merged kommunes
df_aggregated = new_df.groupby("kommune_nr", as_index=False).agg({"value": "sum"})

In [ ]:
df_aggregated.head()

In [ ]:
df2.head()

In [ ]:
df1.head()

In [ ]:
df3 = df_aggregated.copy()

In [ ]:
kommune_pop_aar = df1.rename(columns={"value": "befolkning"})
kommune_pop_fjor = df3.rename(columns={"value": "befolkningx"})

In [ ]:
kommune_befolk = pd.merge(
    kommune_pop_aar, kommune_pop_fjor, on="kommune_nr", how="inner"
)

In [ ]:
kommune_befolk["befolkning_delta"] = (
    kommune_befolk["befolkning"] / kommune_befolk["befolkningx"]
)

In [ ]:
kommune_befolk = kommune_befolk.rename(columns={"kommune_nr": "b_kommunenr"})

kommune_befolk = kommune_befolk[["b_kommunenr", "befolkning_delta"]]

kommune_befolk

In [ ]:
import pyjstat
from pyjstat import pyjstat

POST_URL = "https://data.ssb.no/api/v0/no/table/06944/"
bef_kom = {
    "query": [
        {
            "code": "Region",
            "selection": {
                "filter": "vs:Kommune",
                "values": [
                    "3101",
                    "3103",
                    "3105",
                    "3107",
                    "3110",
                    "3112",
                    "3114",
                    "3116",
                    "3118",
                    "3120",
                    "3122",
                    "3124",
                    "3201",
                    "3203",
                    "3205",
                    "3207",
                    "3209",
                    "3212",
                    "3214",
                    "3216",
                    "3218",
                    "3220",
                    "3222",
                    "3224",
                    "3226",
                    "3228",
                    "3230",
                    "3232",
                    "3234",
                    "3236",
                    "3238",
                    "3240",
                    "3242",
                    "3001",
                    "3002",
                    "3003",
                    "3004",
                    "3005",
                    "3006",
                    "3007",
                    "3011",
                    "3012",
                    "3013",
                    "3014",
                    "3015",
                    "3016",
                    "3017",
                    "3018",
                    "3019",
                    "3020",
                    "3021",
                    "3022",
                    "3023",
                    "3024",
                    "3025",
                    "3026",
                    "3027",
                    "3028",
                    "3029",
                    "3030",
                    "3031",
                    "3032",
                    "3033",
                    "3034",
                    "3035",
                    "3036",
                    "3037",
                    "3038",
                    "3039",
                    "3040",
                    "3041",
                    "3042",
                    "3043",
                    "3044",
                    "3045",
                    "3046",
                    "3047",
                    "3048",
                    "3049",
                    "3050",
                    "3051",
                    "3052",
                    "3053",
                    "3054",
                    "0101",
                    "0102",
                    "0103",
                    "0104",
                    "0105",
                    "0106",
                    "0111",
                    "0113",
                    "0114",
                    "0115",
                    "0116",
                    "0117",
                    "0118",
                    "0119",
                    "0121",
                    "0122",
                    "0123",
                    "0124",
                    "0125",
                    "0127",
                    "0128",
                    "0130",
                    "0131",
                    "0133",
                    "0134",
                    "0135",
                    "0136",
                    "0137",
                    "0138",
                    "0199",
                    "0211",
                    "0213",
                    "0214",
                    "0215",
                    "0216",
                    "0217",
                    "0219",
                    "0220",
                    "0221",
                    "0226",
                    "0227",
                    "0228",
                    "0229",
                    "0230",
                    "0231",
                    "0233",
                    "0234",
                    "0235",
                    "0236",
                    "0237",
                    "0238",
                    "0239",
                    "0299",
                    "0601",
                    "0602",
                    "0604",
                    "0605",
                    "0612",
                    "0615",
                    "0616",
                    "0617",
                    "0618",
                    "0619",
                    "0620",
                    "0621",
                    "0622",
                    "0623",
                    "0624",
                    "0625",
                    "0626",
                    "0627",
                    "0628",
                    "0631",
                    "0632",
                    "0633",
                    "0699",
                    "0301",
                    "0399",
                    "3301",
                    "3303",
                    "3305",
                    "3310",
                    "3312",
                    "3314",
                    "3316",
                    "3318",
                    "3320",
                    "3322",
                    "3324",
                    "3326",
                    "3328",
                    "3330",
                    "3332",
                    "3334",
                    "3336",
                    "3338",
                    "3401",
                    "3403",
                    "3405",
                    "3407",
                    "3411",
                    "3412",
                    "3413",
                    "3414",
                    "3415",
                    "3416",
                    "3417",
                    "3418",
                    "3419",
                    "3420",
                    "3421",
                    "3422",
                    "3423",
                    "3424",
                    "3425",
                    "3426",
                    "3427",
                    "3428",
                    "3429",
                    "3430",
                    "3431",
                    "3432",
                    "3433",
                    "3434",
                    "3435",
                    "3436",
                    "3437",
                    "3438",
                    "3439",
                    "3440",
                    "3441",
                    "3442",
                    "3443",
                    "3446",
                    "3447",
                    "3448",
                    "3449",
                    "3450",
                    "3451",
                    "3452",
                    "3453",
                    "3454",
                    "0401",
                    "0402",
                    "0403",
                    "0412",
                    "0414",
                    "0415",
                    "0417",
                    "0418",
                    "0419",
                    "0420",
                    "0423",
                    "0425",
                    "0426",
                    "0427",
                    "0428",
                    "0429",
                    "0430",
                    "0432",
                    "0434",
                    "0435",
                    "0436",
                    "0437",
                    "0438",
                    "0439",
                    "0441",
                    "0499",
                    "0501",
                    "0502",
                    "0511",
                    "0512",
                    "0513",
                    "0514",
                    "0515",
                    "0516",
                    "0517",
                    "0518",
                    "0519",
                    "0520",
                    "0521",
                    "0522",
                    "0528",
                    "0529",
                    "0532",
                    "0533",
                    "0534",
                    "0536",
                    "0538",
                    "0540",
                    "0541",
                    "0542",
                    "0543",
                    "0544",
                    "0545",
                    "0599",
                    "3801",
                    "3802",
                    "3803",
                    "3804",
                    "3805",
                    "3806",
                    "3807",
                    "3808",
                    "3811",
                    "3812",
                    "3813",
                    "3814",
                    "3815",
                    "3816",
                    "3817",
                    "3818",
                    "3819",
                    "3820",
                    "3821",
                    "3822",
                    "3823",
                    "3824",
                    "3825",
                    "3901",
                    "3903",
                    "3905",
                    "3907",
                    "3909",
                    "3911",
                    "4001",
                    "4003",
                    "4005",
                    "4010",
                    "4012",
                    "4014",
                    "4016",
                    "4018",
                    "4020",
                    "4022",
                    "4024",
                    "4026",
                    "4028",
                    "4030",
                    "4032",
                    "4034",
                    "4036",
                    "0701",
                    "0702",
                    "0703",
                    "0704",
                    "0705",
                    "0706",
                    "0707",
                    "0708",
                    "0709",
                    "0710",
                    "0711",
                    "0712",
                    "0713",
                    "0714",
                    "0715",
                    "0716",
                    "0716u",
                    "0717",
                    "0718",
                    "0719",
                    "0720",
                    "0721",
                    "0722",
                    "0723",
                    "0724",
                    "0725",
                    "0726",
                    "0727",
                    "0728",
                    "0729",
                    "0799",
                    "0805",
                    "0806",
                    "0807",
                    "0811",
                    "0814",
                    "0815",
                    "0817",
                    "0819",
                    "0821",
                    "0822",
                    "0826",
                    "0827",
                    "0828",
                    "0829",
                    "0830",
                    "0831",
                    "0833",
                    "0834",
                    "0899",
                    "4201",
                    "4202",
                    "4203",
                    "4204",
                    "4205",
                    "4206",
                    "4207",
                    "4211",
                    "4212",
                    "4213",
                    "4214",
                    "4215",
                    "4216",
                    "4217",
                    "4218",
                    "4219",
                    "4220",
                    "4221",
                    "4222",
                    "4223",
                    "4224",
                    "4225",
                    "4226",
                    "4227",
                    "4228",
                    "0901",
                    "0903",
                    "0904",
                    "0906",
                    "0911",
                    "0912",
                    "0914",
                    "0918",
                    "0919",
                    "0920",
                    "0921",
                    "0922",
                    "0923",
                    "0924",
                    "0926",
                    "0928",
                    "0929",
                    "0932",
                    "0933",
                    "0935",
                    "0937",
                    "0938",
                    "0940",
                    "0941",
                    "0999",
                    "1001",
                    "1002",
                    "1003",
                    "1004",
                    "1014",
                    "1017",
                    "1018",
                    "1021",
                    "1026",
                    "1027",
                    "1029",
                    "1032",
                    "1034",
                    "1037",
                    "1046",
                    "1099",
                    "1101",
                    "1102",
                    "1103",
                    "1106",
                    "1108",
                    "1111",
                    "1112",
                    "1114",
                    "1119",
                    "1120",
                    "1121",
                    "1122",
                    "1124",
                    "1127",
                    "1129",
                    "1130",
                    "1133",
                    "1134",
                    "1135",
                    "1141",
                    "1142",
                    "1144",
                    "1145",
                    "1146",
                    "1149",
                    "1151",
                    "1154",
                    "1159",
                    "1160",
                    "1199",
                    "4601",
                    "4602",
                    "4611",
                    "4612",
                    "4613",
                    "4614",
                    "4615",
                    "4616",
                    "4617",
                    "4618",
                    "4619",
                    "4620",
                    "4621",
                    "4622",
                    "4623",
                    "4624",
                    "4625",
                    "4626",
                    "4627",
                    "4628",
                    "4629",
                    "4630",
                    "4631",
                    "4632",
                    "4633",
                    "4634",
                    "4635",
                    "4636",
                    "4637",
                    "4638",
                    "4639",
                    "4640",
                    "4641",
                    "4642",
                    "4643",
                    "4644",
                    "4645",
                    "4646",
                    "4647",
                    "4648",
                    "4649",
                    "4650",
                    "4651",
                    "1201",
                    "1211",
                    "1214",
                    "1216",
                    "1219",
                    "1221",
                    "1222",
                    "1223",
                    "1224",
                    "1227",
                    "1228",
                    "1230",
                    "1231",
                    "1232",
                    "1233",
                    "1234",
                    "1235",
                    "1238",
                    "1241",
                    "1242",
                    "1243",
                    "1244",
                    "1245",
                    "1246",
                    "1247",
                    "1248",
                    "1249",
                    "1250",
                    "1251",
                    "1252",
                    "1253",
                    "1255",
                    "1256",
                    "1259",
                    "1260",
                    "1263",
                    "1264",
                    "1265",
                    "1266",
                    "1299",
                    "1301",
                    "1401",
                    "1411",
                    "1412",
                    "1413",
                    "1416",
                    "1417",
                    "1418",
                    "1419",
                    "1420",
                    "1421",
                    "1422",
                    "1424",
                    "1426",
                    "1428",
                    "1429",
                    "1430",
                    "1431",
                    "1432",
                    "1433",
                    "1438",
                    "1439",
                    "1441",
                    "1443",
                    "1444",
                    "1445",
                    "1448",
                    "1449",
                    "1499",
                    "1501",
                    "1502",
                    "1503",
                    "1504",
                    "1505",
                    "1506",
                    "1507",
                    "1508",
                    "1511",
                    "1514",
                    "1515",
                    "1516",
                    "1517",
                    "1519",
                    "1520",
                    "1523",
                    "1524",
                    "1525",
                    "1526",
                    "1527",
                    "1528",
                    "1529",
                    "1531",
                    "1532",
                    "1534",
                    "1535",
                    "1539",
                    "1543",
                    "1545",
                    "1546",
                    "1547",
                    "1548",
                    "1551",
                    "1554",
                    "1556",
                    "1557",
                    "1560",
                    "1563",
                    "1566",
                    "1567",
                    "1569",
                    "1571",
                    "1572",
                    "1573",
                    "1576",
                    "1577",
                    "1578",
                    "1579",
                    "1580",
                    "1599",
                    "5001",
                    "5004",
                    "5005",
                    "5006",
                    "5007",
                    "5011",
                    "5012",
                    "5013",
                    "5014",
                    "5015",
                    "5016",
                    "5017",
                    "5018",
                    "5019",
                    "5020",
                    "5021",
                    "5022",
                    "5023",
                    "5024",
                    "5025",
                    "5026",
                    "5027",
                    "5028",
                    "5029",
                    "5030",
                    "5031",
                    "5032",
                    "5033",
                    "5034",
                    "5035",
                    "5036",
                    "5037",
                    "5038",
                    "5039",
                    "5040",
                    "5041",
                    "5042",
                    "5043",
                    "5044",
                    "5045",
                    "5046",
                    "5047",
                    "5048",
                    "5049",
                    "5050",
                    "5051",
                    "5052",
                    "5053",
                    "5054",
                    "5055",
                    "5056",
                    "5057",
                    "5058",
                    "5059",
                    "5060",
                    "5061",
                    "1601",
                    "1612",
                    "1613",
                    "1617",
                    "1620",
                    "1621",
                    "1622",
                    "1624",
                    "1627",
                    "1630",
                    "1632",
                    "1633",
                    "1634",
                    "1635",
                    "1636",
                    "1638",
                    "1640",
                    "1644",
                    "1645",
                    "1648",
                    "1653",
                    "1657",
                    "1662",
                    "1663",
                    "1664",
                    "1665",
                    "1699",
                    "1702",
                    "1703",
                    "1711",
                    "1714",
                    "1717",
                    "1718",
                    "1719",
                    "1721",
                    "1723",
                    "1724",
                    "1725",
                    "1729",
                    "1736",
                    "1738",
                    "1739",
                    "1740",
                    "1742",
                    "1743",
                    "1744",
                    "1748",
                    "1749",
                    "1750",
                    "1751",
                    "1755",
                    "1756",
                    "1799",
                    "1804",
                    "1805",
                    "1806",
                    "1811",
                    "1812",
                    "1813",
                    "1814",
                    "1815",
                    "1816",
                    "1818",
                    "1820",
                    "1822",
                    "1824",
                    "1825",
                    "1826",
                    "1827",
                    "1828",
                    "1832",
                    "1833",
                    "1834",
                    "1835",
                    "1836",
                    "1837",
                    "1838",
                    "1839",
                    "1840",
                    "1841",
                    "1842",
                    "1843",
                    "1845",
                    "1848",
                    "1849",
                    "1850",
                    "1851",
                    "1852",
                    "1853",
                    "1854",
                    "1855",
                    "1856",
                    "1857",
                    "1858",
                    "1859",
                    "1860",
                    "1865",
                    "1866",
                    "1867",
                    "1868",
                    "1870",
                    "1871",
                    "1874",
                    "1875",
                    "1899",
                    "5501",
                    "5503",
                    "5510",
                    "5512",
                    "5514",
                    "5516",
                    "5518",
                    "5520",
                    "5522",
                    "5524",
                    "5526",
                    "5528",
                    "5530",
                    "5532",
                    "5534",
                    "5536",
                    "5538",
                    "5540",
                    "5542",
                    "5544",
                    "5546",
                    "5601",
                    "5603",
                    "5605",
                    "5607",
                    "5610",
                    "5612",
                    "5614",
                    "5616",
                    "5618",
                    "5620",
                    "5622",
                    "5624",
                    "5626",
                    "5628",
                    "5630",
                    "5632",
                    "5634",
                    "5636",
                    "5401",
                    "5402",
                    "5403",
                    "5404",
                    "5405",
                    "5406",
                    "5411",
                    "5412",
                    "5413",
                    "5414",
                    "5415",
                    "5416",
                    "5417",
                    "5418",
                    "5419",
                    "5420",
                    "5421",
                    "5422",
                    "5423",
                    "5424",
                    "5425",
                    "5426",
                    "5427",
                    "5428",
                    "5429",
                    "5430",
                    "5432",
                    "5433",
                    "5434",
                    "5435",
                    "5436",
                    "5437",
                    "5438",
                    "5439",
                    "5440",
                    "5441",
                    "5442",
                    "5443",
                    "5444",
                    "1901",
                    "1902",
                    "1903",
                    "1911",
                    "1913",
                    "1915",
                    "1917",
                    "1919",
                    "1920",
                    "1921",
                    "1922",
                    "1923",
                    "1924",
                    "1925",
                    "1926",
                    "1927",
                    "1928",
                    "1929",
                    "1931",
                    "1933",
                    "1936",
                    "1938",
                    "1939",
                    "1940",
                    "1941",
                    "1942",
                    "1943",
                    "1999",
                    "2001",
                    "2002",
                    "2003",
                    "2004",
                    "2011",
                    "2012",
                    "2014",
                    "2015",
                    "2016",
                    "2017",
                    "2018",
                    "2019",
                    "2020",
                    "2021",
                    "2022",
                    "2023",
                    "2024",
                    "2025",
                    "2027",
                    "2028",
                    "2030",
                    "2099",
                    "2111",
                    "2112",
                    "2115",
                    "2121",
                    "2131",
                    "2199",
                    "2211",
                    "2299",
                    "2300",
                    "2311",
                    "2321",
                    "2399",
                    "2599",
                    "9999",
                ],
            },
        },
        {"code": "HusholdType", "selection": {"filter": "item", "values": ["0000"]}},
        {
            "code": "ContentsCode",
            "selection": {"filter": "item", "values": ["SamletInntekt"]},
        },
        {"code": "Tid", "selection": {"filter": "item", "values": [year]}},
    ],
    "response": {"format": "json-stat2"},
}

# Behandler spørringen
resultat1 = requests.post(POST_URL, json=bef_kom)

# Check if request was successful
if resultat1.status_code == 200:
    from pyjstat import pyjstat

    # Convert JSON response to DataFrame
    dataset1 = pyjstat.Dataset.read(resultat1.text)
    df = dataset1.write("dataframe")

    # Add 'region' column based on the values in the query
    regions = bef_kom["query"][0]["selection"]["values"]
    df["kommune_nr"] = regions

    # Reorder columns
    df = df[["region", "value", "år", "kommune_nr"]]

In [ ]:
df["value"].value_counts()